# Сдача домашек

* https://t.me/vdobrovolskii
* Дедлайн до 4 октября включительно, далее сдавать можно с -1 баллом в неделю вплоть до "несгораемой суммы" в 3 балла.

# Настройка
Все зависимости перечислены в ячейке ниже. Кроме того, есть ещё дополнительные данные (opencorpora, например). Они тоже скачиваются в первых ячейках.

In [ ]:
%%writefile requirements.txt
nltk>=3.4.5
razdel>=0.4.0
rusenttokenize>=0.0.5
b-labs-models>=2017.8.22
lxml>=4.2.1
spacy>=2.1.4
pymystem3>=0.2.0
rnnmorph>=0.4.0
protobuf<3.20.0

Writing requirements.txt


In [ ]:
!pip install --user --upgrade --force-reinstall -r requirements.txt

In [ ]:
!pip install keras==2.1.4 tensorflow==1.15.5

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!python -m spacy download en_core_web_sm

### Дополнительные данные

Opencorpora: 31 Мб по сети, 530 Мб в распакованном виде

In [ ]:
!wget http://opencorpora.org/files/export/annot/annot.opcorpora.xml.bz2

--2022-10-02 15:52:13--  http://opencorpora.org/files/export/annot/annot.opcorpora.xml.bz2
Resolving opencorpora.org (opencorpora.org)... 164.92.197.18
Connecting to opencorpora.org (opencorpora.org)|164.92.197.18|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32754997 (31M) [application/x-bzip2]
Saving to: ‘annot.opcorpora.xml.bz2’

annot.opcorpora.xml 100%[===================>]  31.24M  19.1MB/s    in 1.6s    

2022-10-02 15:52:14 (19.1 MB/s) - ‘annot.opcorpora.xml.bz2’ saved [32754997/32754997]



In [ ]:
!bzip2 -d annot.opcorpora.xml.bz2

bzip2: Output file annot.opcorpora.xml already exists.


In [ ]:
!ls

annot.opcorpora.xml  requirements.txt  sample_data


### Тестовые примеры

In [ ]:
example1 = "this's a sent tokenize test. this is sent two. is this sent three? sent 4 is cool! Now it’s your turn."
example2 = """
    An ambitious campus expansion plan was proposed by Fr. Vernon F. Gallagher in 1952.
    Assumption Hall, the first student dormitory, was opened in 1954,
    and Rockwell Hall was dedicated in November 1958, housing the schools of business and law.
    It was during the tenure of F. Henry J. McAnulty that Fr. Gallagher's ambitious plans were put to action.
"""
example3 = """
    А что насчёт русского языка? Хорошо ли сегментируются имена?
    Ай да А.С. Пушкин! Ай да сукин сын!
    «Как же так?! Захар...» — воскликнут Пронин.
    - "Так в чем же дело?" - "Не ра-ду-ют".
    И т. д. и т. п. В общем, вся газета.
    Православие... более всего подходит на роль такой идеи...
    Нефть за $27/барр. не снится.
"""
example4 = """
    Кружка-термос на 0.5л (50/64 см³, 516;...) стоит $3.88
"""
example5 = """
    Good muffins cost $3.88 in New York.  Please buy me two of them. Thanks.
"""

# Сегментация предложений
Первая задача - разбиение текста на предложения

### Экперименты

##### NLTK - Natural Language Toolkit
Популярная платформа для анализа текстов. Особенно хорошо работает для английского. В основном не содержит ничего из машинного обучения, только старые добрые правила.

In [ ]:
from nltk.tokenize import sent_tokenize
sent_tokenize(example1)

["this's a sent tokenize test.",
 'this is sent two.',
 'is this sent three?',
 'sent 4 is cool!',
 'Now it’s your turn.']

А вот тут что-то пошло не так

In [ ]:
sent_tokenize(example2)

['\n    An ambitious campus expansion plan was proposed by Fr.',
 'Vernon F. Gallagher in 1952.',
 'Assumption Hall, the first student dormitory, was opened in 1954,\n    and Rockwell Hall was dedicated in November 1958, housing the schools of business and law.',
 'It was during the tenure of F. Henry J. McAnulty that Fr.',
 "Gallagher's ambitious plans were put to action."]

А что насчёт русского языка?

In [ ]:
sent_tokenize(example3)

['\n    А что насчёт русского языка?',
 'Хорошо ли сегментируются имена?',
 'Ай да А.С.',
 'Пушкин!',
 'Ай да сукин сын!',
 '«Как же так?!',
 'Захар...» — воскликнут Пронин.',
 '- "Так в чем же дело?"',
 '- "Не ра-ду-ют".',
 'И т. д. и т. п. В общем, вся газета.',
 'Православие... более всего подходит на роль такой идеи...\n    Нефть за $27/барр.',
 'не снится.']

https://github.com/Mottl/ru_punkt

Data for sentence tokenization was taken from 3 sources:

  * Articles from Russian Wikipedia (about 1 million sentences)
  * Common Russian abbreviations from Russian orthographic dictionary, edited by V. V. Lopatin;
  * Generated names initials.

In [ ]:
sent_tokenize(example3, language="russian")

['\n    А что насчёт русского языка?',
 'Хорошо ли сегментируются имена?',
 'Ай да А.С. Пушкин!',
 'Ай да сукин сын!',
 '«Как же так?!',
 'Захар...» — воскликнут Пронин.',
 '- "Так в чем же дело?"',
 '- "Не ра-ду-ют".',
 'И т. д. и т. п. В общем, вся газета.',
 'Православие... более всего подходит на роль такой идеи...\n    Нефть за $27/барр.',
 'не снится.']

https://github.com/natasha/razdel

razdel старается разбивать текст на предложения и токены так, как это сделано в 4 датасетах: SynTagRus, OpenCorpora, ГИКРЯ и РНК из репозитория morphoRuEval-2017.

В основном это новостные тексты и литература. Правила razdel заточены под них.

На текстах другой тематики (социальные сети, научные статьи) библиотека может работать хуже.

In [ ]:
from razdel import sentenize
list(sentenize(example3))

[Substring(5, 33, 'А что насчёт русского языка?'),
 Substring(34, 65, 'Хорошо ли сегментируются имена?'),
 Substring(70, 88, 'Ай да А.С. Пушкин!'),
 Substring(89, 105, 'Ай да сукин сын!'),
 Substring(110, 123, '«Как же так?!'),
 Substring(124, 154, 'Захар...» — воскликнут Пронин.'),
 Substring(159, 181, '- "Так в чем же дело?"'),
 Substring(182, 198, '- "Не ра-ду-ют".'),
 Substring(203, 218, 'И т. д. и т. п.'),
 Substring(219, 239, 'В общем, вся газета.'),
 Substring(244,
           301,
           'Православие... более всего подходит на роль такой идеи...'),
 Substring(306, 335, 'Нефть за $27/барр. не снится.')]

https://github.com/deepmipt/ru_sentence_tokenizer
    
A simple and fast rule-based sentence segmentation. Tested on OpenCorpora and SynTagRus datasets.

In [ ]:
from rusenttokenize import ru_sent_tokenize
ru_sent_tokenize(example3)

['А что насчёт русского языка?',
 'Хорошо ли сегментируются имена?',
 'Ай да А.С. Пушкин!',
 'Ай да сукин сын!',
 '«Как же так?!',
 'Захар...» — воскликнут Пронин.',
 '- "Так в чем же дело?"',
 '- "Не ра-ду-ют".',
 'И т. д. и т. п.',
 'В общем, вся газета.',
 'Православие... более всего подходит на роль такой идеи...',
 'Нефть за $27/барр. не снится.',
 '']

### Бенчмарки
Много вариантов... Нужно измерять

In [ ]:
# WARNING: RAM bound task, XML parsing is expensive
# Similar to https://github.com/deepmipt/ru_sentence_tokenizer/blob/master/metrics/calculate.ipynb
import re
from lxml import etree

# \W -> Any non-word character
RE_ENDS_WITH_PUNCT = re.compile(r".*\W$")

OPENCORPORA_FILE = "annot.opcorpora.xml"
sentences = list(etree.parse(OPENCORPORA_FILE).xpath('//source/text()'))
singles = []
compounds = []
s2 = sentences.pop().strip()
singles.append(s2)
while sentences:
    s1 = sentences.pop().strip()
    singles.append(s1)
    if RE_ENDS_WITH_PUNCT.match(s1) and not s1.endswith(':') and not s2.startswith('—'):
        compounds.append((s1, s2))
    s2 = s1
        
print(f'Read {len(singles)} sentences from {OPENCORPORA_FILE}')
        
del sentences

Read 110304 sentences from annot.opcorpora.xml


In [ ]:
compounds[:10]

[('• Четвёртая точка (IV), открыта при наблюдении с высотных самолётов.',
  'Располагается на 20° ниже противосолнечной точки.'),
 ('• Точка Брюстера (Br), открыта Брюстером в 1840 г., расположена на 15°—20° ниже Cолнца, но её трудно наблюдать из-за близости солнечного диска.',
  '• Четвёртая точка (IV), открыта при наблюдении с высотных самолётов.'),
 ('• Точка Бабинэ (Ba), открыта Бабинэ в 1840 г., расположена на 15°—20° выше Cолнца, но её трудно наблюдать из-за близости солнечного диска.',
  '• Точка Брюстера (Br), открыта Брюстером в 1840 г., расположена на 15°—20° ниже Cолнца, но её трудно наблюдать из-за близости солнечного диска.'),
 ('Поэтому по её положению определяют степень затуманенности атмосферы.',
  '• Точка Бабинэ (Ba), открыта Бабинэ в 1840 г., расположена на 15°—20° выше Cолнца, но её трудно наблюдать из-за близости солнечного диска.'),
 ('• Точка Араго (А), названная в честь открывателя, это точка, которая в воздухе без дымки и тумана расположена на 20° выше противос

In [ ]:
def check_sent_tokenizer(tokenizer, singles, compounds):
    correct_count_in_singles = 0
    for sentence in singles:
        correct_count_in_singles += len(tokenizer(sentence)) == 1

    correct_count_in_compounds = 0
    for s1, s2 in compounds:
        correct_count_in_compounds += tokenizer(s1 + ' ' + s2) == [s1, s2]

    return (correct_count_in_singles / len(singles), correct_count_in_compounds / len(compounds))

In [ ]:
from nltk.tokenize import sent_tokenize
%time singles_score, compounds_score = check_sent_tokenizer(sent_tokenize, singles, compounds)
print(f'sent_tokenizer scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

CPU times: user 9.2 s, sys: 54.7 ms, total: 9.25 s
Wall time: 9.45 s
sent_tokenizer scores: 94.30%, 86.07%


In [ ]:
russian_sent_tokenize = lambda s : sent_tokenize(s, language="russian")
%time singles_score, compounds_score = check_sent_tokenizer(russian_sent_tokenize, singles, compounds)
print(f'russian sent_tokenizer scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

CPU times: user 9.04 s, sys: 37.9 ms, total: 9.08 s
Wall time: 9.12 s
russian sent_tokenizer scores: 96.78%, 88.85%


In [ ]:
from razdel import sentenize
razdel_sent_tokenize = lambda text : [s.text for s in sentenize(text)]
%time singles_score, compounds_score = check_sent_tokenizer(razdel_sent_tokenize, singles, compounds)
print(f'razdel scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

CPU times: user 7.05 s, sys: 35.5 ms, total: 7.09 s
Wall time: 7.11 s
razdel scores: 99.07%, 95.39%


In [ ]:
from rusenttokenize import ru_sent_tokenize
deepmipt_sent_tokenize = ru_sent_tokenize
%time singles_score, compounds_score = check_sent_tokenizer(deepmipt_sent_tokenize, singles, compounds)
print(f'deepmipt scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

CPU times: user 6.76 s, sys: 23 ms, total: 6.79 s
Wall time: 6.82 s
deepmipt scores: 98.73%, 93.42%


Аналогичные бенчмарки:
- https://github.com/natasha/razdel/blob/master/eval.ipynb
- https://github.com/deepmipt/ru_sentence_tokenizer/blob/master/metrics/calculate.ipynb

### Задание 1: "Кирпич"
Скачайте предложенный текст. Найдите первое предложение, которое отличается в разбиениях, порождённых rusenttokenize и razdel. Верните номер этого предложения.

In [ ]:
!wget https://www.dropbox.com/s/q5wo34gfbepc7am/htbg.txt

--2022-10-02 15:50:45--  https://www.dropbox.com/s/q5wo34gfbepc7am/htbg.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/q5wo34gfbepc7am/htbg.txt [following]
--2022-10-02 15:50:45--  https://www.dropbox.com/s/raw/q5wo34gfbepc7am/htbg.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6cd67b07c5e2046b7ff12ca4a5.dl.dropboxusercontent.com/cd/0/inline/BuBnt0qYduVELCNzI_WoCdQmDLjVoCqkkugTvpzDSCpRnUwIJtS03bk299lLfw3tB03r3P1Gz78n_O2mfkMLzZXFwQHd4J8Wj8OJlrVUvuAZzN-p0nRIzuL6N7PEYBOFSZj4TH4VeOPzDndZa7DbJHFPt3gM3TWOeL0YtVaYK0eaSg/file# [following]
--2022-10-02 15:50:46--  https://uc6cd67b07c5e2046b7ff12ca4a5.dl.dropboxusercontent.com/cd/0/inline/BuBnt0qYduVELCNzI_WoCdQmDLjVoCqkkugTvpzDSCpRnUwIJtS03bk299lLfw3tB03r3P1Gz78n_O2mfkMLzZXFwQHd

In [ ]:
!head htbg.txt

 Аркадий и Борис Стругацкие
 Трудно быть богом


 «То были дни, когда я познал, что значит: страдать; что значит: стыдиться; что значит: отчаяться».

    Пьер Абеляр





In [ ]:
list(ru_sent_tokenize("Привет. Тест"))

['Привет.', 'Тест']

In [ ]:
from razdel import sentenize
from rusenttokenize import ru_sent_tokenize


with open("htbg.txt", "r") as f:
    text = f.read()
    
def get_first_different_sentence(text: str) -> int:
    # your code here
    razdel_sentences = sentenize(text)
    ru_sentences = ru_sent_tokenize(text)
    for i, sent in enumerate(zip(razdel_sentences, ru_sentences)):
      if sent[0].text != sent[1]:
        return i
    return -1

assert get_first_different_sentence(text) == 329

### Задание 2: Lazy baseline
Напишите свой sent_tokenize, который будет делить предложения только по точкам, восклицательным и вопросительным знакам. Измерьте для него время работы и метрики на opencorpora.

In [ ]:
import re
from typing import List

from nltk.tokenize import sent_tokenize
 

def my_sent_tokenize(text: str):
    # your code here
    return [s.strip() for s in re.findall(r'[^?.!]+(?:[.!?]|$)', text)]

assert my_sent_tokenize(example1) == sent_tokenize(example1)
%time singles_score, compounds_score = check_sent_tokenizer(my_sent_tokenize, singles, compounds)
assert singles_score >= 0.85
print(f'your scores: {singles_score*100:.2f}%, {compounds_score*100:.2f}%')

CPU times: user 800 ms, sys: 0 ns, total: 800 ms
Wall time: 809 ms
your scores: 90.98%, 80.05%


In [ ]:
my_sent_tokenize("Привет!!! Пока")

['Привет!', 'Пока']

# Токенизация

Самый наивный способ токенизировать текст -- разделить с помощью split. Но split упускает очень много всего, например, банально не отделяет пунктуацию от слов. Кроме этого, есть ещё много менее тривиальных проблем. Поэтому лучше использовать готовые токенизаторы.

In [ ]:
from nltk.tokenize import word_tokenize
print(word_tokenize(example5))

['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']


In [ ]:
from nltk import tokenize
dir(tokenize)[:19]

['BlanklineTokenizer',
 'LegalitySyllableTokenizer',
 'LineTokenizer',
 'MWETokenizer',
 'NLTKWordTokenizer',
 'PunktSentenceTokenizer',
 'RegexpTokenizer',
 'ReppTokenizer',
 'SExprTokenizer',
 'SpaceTokenizer',
 'StanfordSegmenter',
 'SyllableTokenizer',
 'TabTokenizer',
 'TextTilingTokenizer',
 'ToktokTokenizer',
 'TreebankWordDetokenizer',
 'TreebankWordTokenizer',
 'TweetTokenizer',
 'WhitespaceTokenizer']

Они умеют выдавать индексы начала и конца каждого токена:

In [ ]:
from nltk import tokenize
wh_tok = tokenize.WhitespaceTokenizer()
print(list(wh_tok.span_tokenize(example5)))
print(wh_tok.tokenize(example5))

[(5, 9), (10, 17), (18, 22), (23, 28), (29, 31), (32, 35), (36, 41), (43, 49), (50, 53), (54, 56), (57, 60), (61, 63), (64, 69), (70, 77)]
['Good', 'muffins', 'cost', '$3.88', 'in', 'New', 'York.', 'Please', 'buy', 'me', 'two', 'of', 'them.', 'Thanks.']


Некторые токенизаторы ведут себя специфично:

In [ ]:
tokenize.TreebankWordTokenizer().tokenize("don't stop me")

['do', "n't", 'stop', 'me']

In [ ]:
import spacy
spacy_nlp = spacy.load('en_core_web_sm')
doc = spacy_nlp(example5, disable=["parser"])
print([token.text for token in doc])

['\n    ', 'Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', ' ', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.', '\n']


In [ ]:
doc = spacy_nlp("don't stop me. I can't do that")
for token in doc:
    print(token.text)

do
n't
stop
me
.
I
ca
n't
do
that


In [ ]:
from nltk.tokenize import word_tokenize
print(word_tokenize(example4))

['Кружка-термос', 'на', '0.5л', '(', '50/64', 'см³', ',', '516', ';', '...', ')', 'стоит', '$', '3.88']


In [ ]:
from razdel import tokenize
list(tokenize(example4))

[Substring(5, 18, 'Кружка-термос'),
 Substring(19, 21, 'на'),
 Substring(22, 25, '0.5'),
 Substring(25, 26, 'л'),
 Substring(27, 28, '('),
 Substring(28, 33, '50/64'),
 Substring(34, 37, 'см³'),
 Substring(37, 38, ','),
 Substring(39, 42, '516'),
 Substring(42, 43, ';'),
 Substring(43, 46, '...'),
 Substring(46, 47, ')'),
 Substring(48, 53, 'стоит'),
 Substring(54, 55, '$'),
 Substring(55, 59, '3.88')]

### Задание 3: Diff
Напишите функцию, которая будет выводить разницу между токенизацией razdel'а и nltk.

In [ ]:
from difflib import SequenceMatcher # USE THIS
from razdel import tokenize
from nltk.tokenize import word_tokenize

with open("htbg.txt", "r") as f:
    text = f.read()
    
def get_tokenization_differences(text: str) -> int:
    differences = []
    # YOUR CODE HERE
    sent1 = [_.text for _ in tokenize(text)]
    sent2 = word_tokenize(text)

    diff = SequenceMatcher(lambda x: x==" ", sent1, sent2)

    for block in diff.get_opcodes():
      if block[0] != 'equal':
        differences.append(block)

    return differences

assert len(get_tokenization_differences(text)) == 613

# Стоп-слова и пунктуация

Стоп-слова - это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конкретном документе, то есть играют роль шума. Поэтому их принято убирать. По той же причине убирают и пунктуацию.

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [ ]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
noise = stopwords.words('russian') + list(punctuation)

In [ ]:
noise

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

### Задание 4: Стоп-слова from scratch
Постройте свой список стоп-слов на основе Opencorpora

In [ ]:
from collections import Counter
import re
from lxml import etree
from nltk.tokenize import word_tokenize

OPENCORPORA_FILE = "annot.opcorpora.xml"
sentences = list(etree.parse(OPENCORPORA_FILE).xpath('//source/text()'))
print(f'Read {len(sentences)} sentences from {OPENCORPORA_FILE}')

words = word_tokenize(''.join(sentences))

# Your code here
def most_frequent(words):
    occurence_count = Counter(words)
    return occurence_count.most_common(100)
   
print(most_frequent(words))
stopwords = most_frequent(words)

del sentences

Read 110304 sentences from annot.opcorpora.xml
[(',', 152785), ('в', 51861), ('и', 51089), ('на', 23709), ('не', 20144), ('«', 18167), ('»', 18096), ('с', 15942), ('что', 15525), ('—', 14694), (')', 14008), ('(', 13047), ('по', 9744), ('.', 8950), (':', 7792), ('как', 7556), ('к', 7326), ('а', 6962), ('из', 6673), ('для', 6564), ('о', 5888), ('от', 5861), ('это', 5567), ('за', 5283), ('его', 5196), ('или', 4910), ('?', 4316), ('но', 3997), ('он', 3927), ('у', 3821), ('то', 3666), ('!', 3584), ('же', 3583), ('-', 3542), ('до', 3403), (';', 3322), ("''", 3281), ('``', 3210), ('так', 3144), ('их', 3126), ('я', 3120), ('только', 3044), ('было', 2892), ('–', 2866), ('уже', 2761), ('года', 2733), ('был', 2677), ('все', 2581), ('также', 2523), ('бы', 2520), ('том', 2378), ('будет', 2365), ('при', 2350), ('время', 2346), ('может', 2273), ('которые', 2200), ('если', 2120), ('они', 2088), ('этом', 2070), ('году', 2048), ('того', 2015), ('быть', 1992), ('мы', 1908), ('чтобы', 1876), ('были', 1876

In [ ]:
stopwords

In [ ]:
my_stopwords =[word[0] for word in stopwords]

my_stopwords

[',',
 'в',
 'и',
 'на',
 'не',
 '«',
 '»',
 'с',
 'что',
 '—',
 ')',
 '(',
 'по',
 '.',
 ':',
 'как',
 'к',
 'а',
 'из',
 'для',
 'о',
 'от',
 'это',
 'за',
 'его',
 'или',
 '?',
 'но',
 'он',
 'у',
 'то',
 '!',
 'же',
 '-',
 'до',
 ';',
 "''",
 '``',
 'так',
 'их',
 'я',
 'только',
 'было',
 '–',
 'уже',
 'года',
 'был',
 'все',
 'также',
 'бы',
 'том',
 'будет',
 'при',
 'время',
 'может',
 'которые',
 'если',
 'они',
 'этом',
 'году',
 'того',
 'быть',
 'мы',
 'чтобы',
 'были',
 'есть',
 'можно',
 'во',
 'более',
 'когда',
 'этого',
 'она',
 'была',
 'после',
 'который',
 'даже',
 'со',
 'России',
 'очень',
 'под',
 'ещё',
 'без',
 '%',
 'чем',
 'ни',
 'всё',
 'Российской',
 'где',
 'является',
 'тем',
 'лет',
 'ли',
 'между',
 'себя',
 'об',
 'её',
 'ее',
 'всех',
 'этот',
 'Федерации']

# Стемминг

In [ ]:
from nltk.stem.snowball import SnowballStemmer 
from razdel import tokenize

stemmer = SnowballStemmer("russian") 
print([stemmer.stem(token.text) for token in tokenize(example3)])

['а', 'что', 'насчет', 'русск', 'язык', '?', 'хорош', 'ли', 'сегментир', 'им', '?', 'а', 'да', 'а', '.', 'с', '.', 'пушкин', '!', 'а', 'да', 'сукин', 'сын', '!', '«', 'как', 'же', 'так', '?!', 'захар', '...', '»', '—', 'воскликнут', 'пронин', '.', '-', '"', 'так', 'в', 'чем', 'же', 'дел', '?', '"', '-', '"', 'не', 'ра-ду-ют', '"', '.', 'и', 'т', '.', 'д', '.', 'и', 'т', '.', 'п', '.', 'в', 'общ', ',', 'вся', 'газет', '.', 'православ', '...', 'бол', 'всег', 'подход', 'на', 'рол', 'так', 'ид', '...', 'нефт', 'за', '$', '27', '/', 'барр', '.', 'не', 'снит', '.']


# Лемматизация и морфологический анализ

Лемматизация – это сведение разных форм одного слова к начальной форме – лемме. Почему это хорошо?
* Мы хотим рассматривать как отдельную фичу каждое слово, а не каждую его отдельную форму.
* Некоторые стоп-слова стоят только в начальной форме, и без лемматизации выкидываем мы только её.

Для русского есть два хороших лемматизатора: mystem и pymorphy. С pymorphy всё сразу понятно.

Но как работать с Mystem:
* Можно скачать mystem и запускать из терминала с разными параметрами
* pymystem3 - обертка для питона, работает медленнее, но это удобно

## Mystem

In [ ]:
from pymystem3 import Mystem
mystem_analyzer = Mystem()

In [ ]:
!chmod +x /root/.local/bin/mystem

In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2022-10-02 15:52:50--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.245, 5.45.205.241, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.245|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cachev2-spb02.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=122 [following]
--2022-10-02 15:52:51--  http://cachev2-spb02.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=122
Resolving cachev2-spb02.cdn.yandex.net (cachev2-spb02.cdn.yandex.net)... 37.140.137.4, 2a02:6b8:0:2221::304
Connecting to cachev2-spb02.cdn.yandex.net (cachev2-spb02.cdn.yandex.net)|37.140.137.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-3.0-linux3.1 100%[===

Мы инициализировали Mystem c дефолтными параметрами. А вообще параметры есть такие:

    mystem_bin - путь к mystem, если их несколько
    grammar_info - нужна ли грамматическая информация или только леммы (по дефолту нужна)
    disambiguation - нужно ли снятие омонимии - дизамбигуация (по дефолту нужна)
    entire_input - нужно ли сохранять в выводе все (пробелы всякие, например), или можно выкинуть (по дефолту оставляется все)

Методы Mystem принимают строку, токенизатор вшит внутри. Можно, конечно, и пословно анализировать, но тогда он не сможет учитывать контекст.

Можно просто лемматизировать текст:

In [ ]:
print(mystem_analyzer.lemmatize(example3))

['\n', '    ', 'а', ' ', 'что', ' ', 'насчет', ' ', 'русский', ' ', 'язык', '? ', 'хорошо', ' ', 'ли', ' ', 'сегментироваться', ' ', 'имя', '?', '\n', '    ', 'ай', ' ', 'да', ' ', 'а', '.', 'с', '. ', 'пушкин', '! ', 'ай', ' ', 'да', ' ', 'сукин', ' ', 'сын', '!', '\n', '    «', 'как', ' ', 'же', ' ', 'так', '?! ', 'захар', '...', '» — ', 'восклицать', ' ', 'пронин', '.', '\n', '    - "', 'так', ' ', 'в', ' ', 'чем', ' ', 'же', ' ', 'дело', '?" - ', '"', 'не', ' ', 'ра', '-', 'ду', '-', 'ют', '"', '.', '\n', '    ', 'и', ' ', 'т', '.', ' ', 'д', '.', ' ', 'и', ' ', 'т', '.', ' ', 'п', '. ', 'в', ' ', 'общий', ', ', 'весь', ' ', 'газета', '.', '\n', '    ', 'православие', '...', ' ', 'много', ' ', 'все', ' ', 'подходить', ' ', 'на', ' ', 'роль', ' ', 'такой', ' ', 'идея', '...', '\n', '    ', 'нефть', ' ', 'за', ' ', '$', '27', '/', 'барра', '.', ' ', 'не', ' ', 'сниться', '.', '\n']


А можно получить грамматическую информацию:

In [ ]:
mystem_analyzer.analyze(example3)

[{'text': '\n'},
 {'text': '    '},
 {'analysis': [{'lex': 'а', 'wt': 0.9822148681, 'gr': 'CONJ='}], 'text': 'А'},
 {'text': ' '},
 {'analysis': [{'lex': 'что',
    'wt': 0.2934446335,
    'gr': 'SPRO,ед,сред,неод=(вин|им)'}],
  'text': 'что'},
 {'text': ' '},
 {'analysis': [{'lex': 'насчет', 'wt': 1, 'gr': 'PR='}], 'text': 'насчёт'},
 {'text': ' '},
 {'analysis': [{'lex': 'русский',
    'wt': 0.9496245384,
    'gr': 'A=(вин,ед,полн,муж,од|род,ед,полн,муж|род,ед,полн,сред)'}],
  'text': 'русского'},
 {'text': ' '},
 {'analysis': [{'lex': 'язык', 'wt': 0.9963353872, 'gr': 'S,муж,неод=род,ед'}],
  'text': 'языка'},
 {'text': '? '},
 {'analysis': [{'lex': 'хорошо', 'wt': 0.0008292704006, 'gr': 'ADV=вводн'}],
  'text': 'Хорошо'},
 {'text': ' '},
 {'analysis': [{'lex': 'ли', 'wt': 0.7719288468, 'gr': 'PART='}],
  'text': 'ли'},
 {'text': ' '},
 {'analysis': [{'lex': 'сегментироваться',
    'wt': 1,
    'gr': 'V,несов,нп=непрош,мн,изъяв,3-л'}],
  'text': 'сегментируются'},
 {'text': ' '},
 {

## Pymorphy

Это модуль на питоне, довольно быстрый и с кучей функций.

In [ ]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

In [ ]:
pymorphy2_analyzer.parse("мою")

[Parse(word='мою', tag=OpencorporaTag('ADJF,Apro femn,sing,accs'), normal_form='мой', score=0.970588, methods_stack=((DictionaryAnalyzer(), 'мою', 2049, 10),)),
 Parse(word='мою', tag=OpencorporaTag('VERB,impf,tran sing,1per,pres,indc'), normal_form='мыть', score=0.029411, methods_stack=((DictionaryAnalyzer(), 'мою', 2074, 1),))]

### Задание 5: Анализ частей речи

Используя pymorphy2, определите топ-10 самых частотных существительных и глаголов в тексте

In [ ]:
from pymorphy2 import MorphAnalyzer
from razdel import tokenize
from collections import Counter

pymorphy2_analyzer = MorphAnalyzer()
with open("htbg.txt", "r") as f:
    text = f.read()
# YOUR CODE HERE
def most_common_parts(text):
  tokens = tokenize(text)
  nouns = []
  verbs = []
  for token in tokens:
    p = pymorphy2_analyzer.parse(token.text)[0]
    if 'VERB' in p.tag:
      verbs.append(p.normal_form)
    elif 'NOUN' in p.tag:
      nouns.append(p.normal_form)

  return Counter(nouns).most_common(10), Counter(verbs).most_common(10)

nouns, verbs = most_common_parts(text)
print(f"Most common nouns: {nouns}")
print(f"Most common verbs: {verbs}")

Most common nouns: [('румат', 699), ('дон', 577), ('рэб', 194), ('человек', 176), ('рука', 161), ('барон', 129), ('глаз', 115), ('король', 111), ('отец', 98), ('антон', 95)]
Most common verbs: [('быть', 531), ('сказать', 506), ('мочь', 147), ('знать', 118), ('подумать', 91), ('спросить', 82), ('стать', 81), ('хотеть', 62), ('говорить', 62), ('пойти', 56)]


## mystem vs. pymorphy

1) Mystem работает невероятно медленно под windows на больших текстах.

2) Снятие омонимии. Mystem умеет снимать омонимию по контексту (хотя не всегда преуспевает), pymorphy2 берет на вход одно слово и соответственно вообще не умеет дизамбигуировать по контексту:


In [ ]:
homonym1 = 'За время обучения я прослушал больше сорока курсов.'
homonym2 = 'Сорока своровала блестящее украшение со стола.'
mystem_analyzer = Mystem() # инициализирую объект с дефолтными параметрами

print(mystem_analyzer.analyze(homonym1)[-5])
print(mystem_analyzer.analyze(homonym2)[0])

{'analysis': [{'lex': 'сорок', 'wt': 0.8710292578, 'gr': 'NUM=(пр|дат|род|твор)'}], 'text': 'сорока'}
{'analysis': [{'lex': 'сорока', 'wt': 0.1210970059, 'gr': 'S,жен,од=им,ед'}], 'text': 'Сорока'}


## Rnnmorph
Обёртка над pymorphy с разрешением омонимии

https://github.com/IlyaGusev/rnnmorph

https://habr.com/ru/post/339954/

In [ ]:
%tensorflow_version 1.x

from rnnmorph.predictor import RNNMorphPredictor
from razdel import tokenize

predictor = RNNMorphPredictor(language="ru")
homonym = "Косил косой косой косой"
print(predictor.predict([t.text for t in tokenize(homonym)])[1])
print(predictor.predict([t.text for t in tokenize(homonym)])[-1])

In [ ]:
!pip freeze | grep keras

In [ ]:
!pip freeze | grep tensorflow

## GramEval-2020

Соревнование по определению морфологических характеристик, определению синтаксических зависимостей и лемматизации. Готовых инструментов не получилось, но весь код всех конкурсантов доступен.
* https://github.com/dialogue-evaluation/GramEval2020

### Задание 6: Формат

Используя стандартные инструменты переведите корпус htbg.txt в формат CoNLL-U.
Используйте следующие колонки: 
    1. Номер предложения в тексте
    2. Токен в том виде, в котором он встретился в тексте
    3. Лемма токена
    4. POS-таг токена
    5. Вектор грамматических значений токена
    6. Целевая метка (сделайте метку везде OUT)

# Regex 101

In [ ]:
import re

#### match
ищет по заданному шаблону в начале строки

In [ ]:
result = re.match('ab+c.', 'abcdefghijkabcabc') # ищем по шаблону 'ab+c.' 
print (result) # совпадение найдено:

In [ ]:
print(result.group(0)) # выводим найденное совпадение

In [ ]:
result = re.match('abc.', 'abdefghijkabcabc')
print(result) # совпадение не найдено

#### search
ищет по всей строке, возвращает только первое найденное совпадение

In [ ]:
result = re.search('ab+c.', 'aefgabchijkabcabc') 
print(result) 

#### findall
возвращает список всех найденных совпадений

In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

Вопросы: 
1) почему нет последнего abc?
2) почему нет abcx?

#### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit = 2) 
print(result)

#### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

#### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

In [ ]:
# Ваш код